In [27]:
import glob
import time
import numpy as np

In [2]:
path_base="/home/francisco/datasets/datasets/sound/voice/stress"

In [3]:
def getLabel(s):
    if s=="TRUE":
        return 1.0
    if s=="FALSE":
        return 0.0
    if s=="PC":
        return 2.0
def isRightLabel(s):
    if s=="TRUE":
        return True
    if s=="FALSE":
        return True
    if s=="PC":
        return False
    return False

In [4]:
data=[]
labels=[]
file_txt=path_base+"/*.csv"
for filepath in glob.glob(file_txt):
    print("Processing file: {}".format(filepath)) 
    with open(filepath) as fp:  
        line = fp.readline()
        head_list=line.split(";")
        #for i,head in enumerate(head_list):
        #    print(i,head)
        cnt = 1
        line = fp.readline()
        while line:
            data_row=[]
            line_list=line.split(";")
            #print("Line {}: {} {} {}".format(cnt, line_list[0], line_list[1], line_list[2]))
            line = fp.readline()
            cnt += 1
            if not isRightLabel(line_list[0]):
                continue
            for i,datum in enumerate(line_list):
                #print(i,head_list[i],"=",datum)
                if i>1:
                    data_row.append(float(datum))
                if i==0:
                    labels.append(getLabel(datum))
            data.append(data_row)
data_np=np.array(data)
labels_np=np.array(labels)
print(data_np.shape)
print(data_np)
print(labels_np.shape)
print(labels_np)

Processing file: /home/francisco/datasets/datasets/sound/voice/stress/Set_Males.csv
Processing file: /home/francisco/datasets/datasets/sound/voice/stress/Set_Females.csv
(50, 68)
[[1.26278e+02 1.40000e-02 2.90000e-02 ... 1.56810e+01 8.70000e-01
  1.20000e-02]
 [1.42901e+02 5.00000e-03 1.90000e-02 ... 1.08340e+01 9.36000e-01
  8.00000e-03]
 [1.17189e+02 5.00000e-03 1.40000e-02 ... 1.17650e+01 9.67000e-01
  6.00000e-03]
 ...
 [1.92269e+02 8.00000e-03 2.20000e-02 ... 1.06000e+01 9.28000e-01
  7.00000e-03]
 [1.67700e+02 5.00000e-03 2.50000e-02 ... 9.56400e+00 9.33000e-01
  9.00000e-03]
 [2.06086e+02 7.00000e-03 1.30000e-02 ... 9.85100e+00 9.22000e-01
  4.60000e-02]]
(50,)
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]


In [5]:
from sklearn.preprocessing import StandardScaler

In [7]:
ss=StandardScaler()
data_ss=ss.fit_transform(data_np)

print(data_ss.shape)
print(data_ss)

(50, 68)
[[-0.69164371  1.33540079  0.94208383 ...  1.05566522 -1.54961754
  -0.52359428]
 [-0.30927853 -0.90687665 -0.1000443  ... -0.26111133  0.01087617
  -0.80815639]
 [-0.90071049 -0.90687665 -0.62110836 ... -0.00818808  0.74383534
  -0.95043744]
 ...
 [ 0.82629293 -0.15945084  0.21259414 ... -0.32468173 -0.17827458
  -0.87929692]
 [ 0.26115245 -0.90687665  0.52523258 ... -0.60613015 -0.06005536
  -0.73701586]
 [ 1.14411401 -0.40859278 -0.72532118 ... -0.52816133 -0.32013765
   1.89518365]]


In [17]:
import numpy as onp
import jax.numpy as np
from jax import grad, jit, vmap, value_and_grad
from jax import random
from jax.experimental import stax
from jax.experimental.stax import (BatchNorm, Conv, Dense, Flatten, Dropout,
                                   Relu, LogSoftmax)

In [9]:
# Import some additional JAX and dataloader helpers
from jax.scipy.special import logsumexp
from jax.experimental import optimizers

In [10]:
#Test if JAX is using CPU or GPU
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)

gpu


In [11]:
# Generate key which is used to generate random numbers
key = random.PRNGKey(1)

In [19]:
num_classes = 2
hidden=25
dropout_rate=0.5
# Conv seems to work with NHWC but it works with NCHW too
init_fun, net = stax.serial(Dense(hidden),
                            Dropout(dropout_rate),
                            Dense(num_classes),
                            Dropout(dropout_rate),
                            Dense(num_classes),
                            LogSoftmax)

In [24]:
input_shape=(-1,)+ data_ss.shape[1:]
output_shape, params = init_fun(key, input_shape)
print("ouput_shape=",output_shape)

ouput_shape= (-1, 2)


In [47]:
def one_hot(x, k, dtype=np.float32):
    """Create a one-hot encoding of x of size k """
    return np.array(x[:, None] == np.arange(k), dtype)

def loss(params, data, targets,key):
    preds = net(params, data,rng=key)
    return -np.sum(preds * targets)

In [48]:
step_size = 1e-3
opt_init, opt_update, get_params = optimizers.adam(step_size)
opt_state = opt_init(params)
num_epochs = 2

In [49]:
@jit
def update(params, x, y, opt_state,key):
    """ Compute the gradient for a batch and update the parameters """
    value, grads = value_and_grad(loss)(params, x, y,key)
    opt_state = opt_update(0, grads, opt_state)
    return get_params(opt_state), opt_state, value

In [50]:
labels_onehot=one_hot(labels_np,num_classes)
print(labels_onehot.shape)
def run_training_loop(num_epochs, opt_state):
    """ Implements a learning loop over epochs. """
    # Get the initial set of parameters
    params = get_params(opt_state)

    # Loop over the training epochs
    for epoch in range(num_epochs):
        start_time = time.time()
        params, opt_state, loss = update(params, data_ss, labels_onehot, opt_state,key)
        epoch_time = time.time() - start_time
        print("Epoch {} | T: {:0.2f} | loss: {:0.3f} ".format(epoch+1, epoch_time,
                                                                    loss))
    return loss

(50, 2)


In [53]:
run_training_loop(200,opt_state)

Epoch 1 | T: 0.00 | loss: 97.390 
Epoch 2 | T: 0.00 | loss: 92.775 
Epoch 3 | T: 0.00 | loss: 86.724 
Epoch 4 | T: 0.00 | loss: 79.923 
Epoch 5 | T: 0.00 | loss: 72.797 
Epoch 6 | T: 0.00 | loss: 65.648 
Epoch 7 | T: 0.00 | loss: 58.700 
Epoch 8 | T: 0.00 | loss: 52.161 
Epoch 9 | T: 0.00 | loss: 46.281 
Epoch 10 | T: 0.00 | loss: 41.202 
Epoch 11 | T: 0.00 | loss: 36.831 
Epoch 12 | T: 0.00 | loss: 33.016 
Epoch 13 | T: 0.00 | loss: 29.690 
Epoch 14 | T: 0.00 | loss: 26.864 
Epoch 15 | T: 0.00 | loss: 24.504 
Epoch 16 | T: 0.00 | loss: 22.483 
Epoch 17 | T: 0.00 | loss: 20.689 
Epoch 18 | T: 0.00 | loss: 19.099 
Epoch 19 | T: 0.00 | loss: 17.756 
Epoch 20 | T: 0.00 | loss: 16.691 
Epoch 21 | T: 0.00 | loss: 15.853 
Epoch 22 | T: 0.00 | loss: 15.162 
Epoch 23 | T: 0.00 | loss: 14.565 
Epoch 24 | T: 0.00 | loss: 14.035 
Epoch 25 | T: 0.00 | loss: 13.562 
Epoch 26 | T: 0.00 | loss: 13.139 
Epoch 27 | T: 0.00 | loss: 12.760 
Epoch 28 | T: 0.00 | loss: 12.421 
Epoch 29 | T: 0.00 | loss: 12

DeviceArray(8.295582, dtype=float32)